In [1]:
import pandas as pd
import numpy as np
!pip install -U -q PyDrive
from pydrive.drive import GoogleDrive
from google.colab import drive

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense

In [3]:
# data_path = '/home/arsh/Jasleen/Spring 2023/NLP/Group Project/Authorship-Attribution-for-Neural-Text-Generation-master/data/'
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = 'drive/MyDrive/NLP/'

In [5]:
data = pd.read_csv(data_path+'input.csv')
data.tail()

,text,class
11721,how much of your body is your own how much of ...,instructgpt
11722,how do you keep a space station clean how do y...,instructgpt
11723,the city where you pay a years rent up front t...,instructgpt
11724,the bbc news app gives you the best of bbc new...,instructgpt
11725,learn how the bbc is working to strengthen tru...,instructgpt


In [14]:
import pandas as pd
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
def extract_discourse_structure(text):
    sentences = sent_tokenize(text)
    num_sentences = len(sentences)
    num_words = len(word_tokenize(text))
    avg_words_per_sentence = num_words / num_sentences

    return [num_sentences, num_words, avg_words_per_sentence]

data['discourse_structure'] = data['text'].apply(extract_discourse_structure)


In [15]:
def extract_pos_statistics(text):
    words = word_tokenize(text)
    pos_tags = nltk.pos_tag(words)
    pos_counts = nltk.FreqDist(tag for word, tag in pos_tags)

    return pos_counts

data['pos_statistics'] = data['text'].apply(extract_pos_statistics)


In [16]:
def calculate_average_sentence_length(text):
    sentences = sent_tokenize(text)
    num_sentences = len(sentences)
    num_words = len(word_tokenize(text))
    avg_words_per_sentence = num_words / num_sentences

    return avg_words_per_sentence

data['avg_sentence_length'] = data['text'].apply(calculate_average_sentence_length)


In [17]:
# type-token ratio (TTR) of the text.
def calculate_ttr(text):
    words = word_tokenize(text)
    num_tokens = len(words)
    num_types = len(set(words))
    ttr = num_types / num_tokens

    return ttr

data['ttr'] = data['text'].apply(calculate_ttr)


In [18]:
def calculate_average_clauses_per_sentence(text):
    sentences = sent_tokenize(text)
    num_sentences = len(sentences)
    num_clauses = 0

    for sentence in sentences:
        clause_count = sentence.count(',') + 1  # Assuming clauses are separated by commas
        num_clauses += clause_count

    avg_clauses_per_sentence = num_clauses / num_sentences

    return avg_clauses_per_sentence

data['avg_clauses_per_sentence'] = data['text'].apply(calculate_average_clauses_per_sentence)


In [19]:
X = data[['discourse_structure', 'pos_statistics', 'avg_sentence_length', 'ttr', 'avg_clauses_per_sentence']]
y = data['class']


In [20]:
X

,discourse_structure,pos_statistics,avg_sentence_length,ttr,avg_clauses_per_sentence
0,"[1, 370, 370.0]","{'NN': 90, 'JJS': 1, 'NNS': 26, 'IN': 32, 'VBD...",370.0,0.837838,1.0
1,"[1, 466, 466.0]","{'NN': 119, 'VBZ': 17, 'TO': 1, 'VB': 10, 'DT'...",466.0,0.851931,1.0
2,"[1, 476, 476.0]","{'NN': 158, 'VBZ': 12, 'NNS': 46, 'IN': 40, 'J...",476.0,0.907563,1.0
3,"[1, 451, 451.0]","{'NN': 81, 'VBG': 11, 'JJ': 48, 'NNS': 31, 'VB...",451.0,0.793792,1.0
4,"[1, 415, 415.0]","{'NN': 96, 'NNS': 39, 'MD': 7, 'VB': 12, 'IN':...",415.0,0.867470,1.0
...,...,...,...,...,...
11721,"[1, 219, 219.0]","{'WRB': 2, 'JJ': 32, 'IN': 27, 'PRP$': 38, 'NN...",219.0,0.264840,1.0
11722,"[1, 296, 296.0]","{'WRB': 2, 'VB': 30, 'PRP': 4, 'DT': 25, 'NN':...",296.0,0.462838,1.0
11723,"[1, 271, 271.0]","{'DT': 27, 'NN': 46, 'WRB': 4, 'PRP': 5, 'VBP'...",271.0,0.420664,1.0
11724,"[1, 291, 291.0]","{'DT': 28, 'JJ': 25, 'NN': 56, 'VBZ': 12, 'PRP...",291.0,0.422680,1.0


In [21]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer

# Convert 'discourse_structure' column to a 2D array
discourse_structure_arr = np.array(X['discourse_structure'].tolist())

# Convert 'pos_statistics' column to a list of dictionaries
pos_statistics_list = X['pos_statistics'].tolist()

# Use DictVectorizer to convert the list of dictionaries to a matrix
vectorizer = DictVectorizer(sparse=False)
pos_statistics_matrix = vectorizer.fit_transform(pos_statistics_list)

# Concatenate the two transformed features
X = np.concatenate((discourse_structure_arr, pos_statistics_matrix, data[['avg_sentence_length', 'ttr', 'avg_clauses_per_sentence']]), axis=1)
# X = data[['discourse_structure', 'pos_statistics', 'avg_sentence_length', 'ttr', 'avg_clauses_per_sentence']]
X

array([[1.00000000e+00, 3.70000000e+02, 3.70000000e+02, ...,
        3.70000000e+02, 8.37837838e-01, 1.00000000e+00],
       [1.00000000e+00, 4.66000000e+02, 4.66000000e+02, ...,
        4.66000000e+02, 8.51931330e-01, 1.00000000e+00],
       [1.00000000e+00, 4.76000000e+02, 4.76000000e+02, ...,
        4.76000000e+02, 9.07563025e-01, 1.00000000e+00],
       ...,
       [1.00000000e+00, 2.71000000e+02, 2.71000000e+02, ...,
        2.71000000e+02, 4.20664207e-01, 1.00000000e+00],
       [1.00000000e+00, 2.91000000e+02, 2.91000000e+02, ...,
        2.91000000e+02, 4.22680412e-01, 1.00000000e+00],
       [1.00000000e+00, 3.34000000e+02, 3.34000000e+02, ...,
        3.34000000e+02, 4.70059880e-01, 1.00000000e+00]])

In [22]:
from sklearn.preprocessing import LabelEncoder

# Encode the target variable y
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

y_encoded

array([0, 0, 0, ..., 7, 7, 7])